In [1]:
import os
import shutil

# Move the LongPlate files to the 1_chosen_data folder

## Train files

In [14]:
source_folder = 'data/Vehicle Registration Plates.v1-original-images.yolov8/train'
destination_folder = 'data/1_chosen_data/train'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

train_images_folder = source_folder + '/images'
train_labels_folder = source_folder + '/labels'
train_images = os.listdir(train_images_folder)
train_labels = os.listdir(train_labels_folder)

print(f"len(train_images) = {len(train_images)}")
print(f"len(train_labels) = {len(train_labels)}")

len(train_images) = 6176
len(train_labels) = 6176


In [21]:
counter = 0
for filename in train_images:
    if filename.startswith('CarLongPlate'):
        print(filename)
        counter += 1
        if counter == 10:
            break

CarLongPlate0_jpg.rf.68e0b9ccbdf0663f58876e02d480971e.jpg
CarLongPlate100_jpg.rf.6067d4cebf8aa075fc1c9b317f824c84.jpg
CarLongPlate102_jpg.rf.3c1ac339e94f9a48376022e310a8e5c7.jpg
CarLongPlate103_jpg.rf.344c8ec636c3fac0667537ff778ffa3a.jpg
CarLongPlate104_jpg.rf.870fdbc5cf820aede6c9b2dfd8319b7a.jpg
CarLongPlate106_jpg.rf.5561c4ab0b29b8e8ba22b23381bfbe1d.jpg
CarLongPlate107_jpg.rf.f672816f379274ac85026b64c585c948.jpg
CarLongPlate108_jpg.rf.534e54495e2a4dfd7ea8c1df359381bc.jpg
CarLongPlate109_jpg.rf.d8b0fcae17838a8f950f0a60fb3d8f3b.jpg
CarLongPlate10_jpg.rf.1125e4c8cd5267366d855b201cdd7951.jpg


In [10]:
train_car_long_plate_images = [image_name for image_name in train_images if image_name.startswith('CarLongPlate')]
train_car_long_plate_labels = [label for label in train_labels if label.startswith('CarLongPlate')]

print(f"len(train_car_long_plate_images) = {len(train_car_long_plate_images)}")
print(f"len(train_car_long_plate_labels) = {len(train_car_long_plate_labels)}")

len(train_car_long_plate_images) = 3436
len(train_car_long_plate_labels) = 3436


In [ ]:
images_destination_folder = destination_folder + '/images'
labels_destination_folder = destination_folder + '/labels'

if not os.path.exists(images_destination_folder):
    os.makedirs(images_destination_folder)
    
if not os.path.exists(labels_destination_folder):
    os.makedirs(labels_destination_folder)

for long_plate_image, long_plate_label in zip(train_car_long_plate_images, train_car_long_plate_labels):
    source_image_path = os.path.join(train_images_folder, long_plate_image)
    destination_image_path = os.path.join(images_destination_folder, long_plate_image)
    
    source_label_path = os.path.join(train_labels_folder, long_plate_label)
    destination_label_path = os.path.join(labels_destination_folder, long_plate_label)
    
    shutil.copy(source_image_path, destination_image_path)
    shutil.copy(source_label_path, destination_label_path)

print("Copy process completed")

In [19]:
long_plate_images = os.listdir(images_destination_folder)
long_plate_labels = os.listdir(labels_destination_folder)

print(f"len(long_plate_images) = {len(long_plate_images)}")
print(f"len(long_plate_labels) = {len(long_plate_labels)}")

len(long_plate_images) = 3436
len(long_plate_labels) = 3436


## Valid files

In [20]:
source_folder = 'data/Vehicle Registration Plates.v1-original-images.yolov8/valid'
destination_folder = 'data/1_chosen_data/valid'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

valid_images_folder = source_folder + '/images'
valid_labels_folder = source_folder + '/labels'
valid_images = os.listdir(valid_images_folder)
valid_labels = os.listdir(valid_labels_folder)

print(f"len(valid_images) = {len(valid_images)}")
print(f"len(valid_labels) = {len(valid_labels)}")

len(valid_images) = 1765
len(valid_labels) = 1765


In [23]:
valid_car_long_plate_images = [image_name for image_name in valid_images if image_name.startswith('CarLongPlate')]
valid_car_long_plate_labels = [label for label in valid_labels if label.startswith('CarLongPlate')]

print(f"len(valid_car_long_plate_images) = {len(valid_car_long_plate_images)}")
print(f"len(valid_car_long_plate_labels) = {len(valid_car_long_plate_labels)}")

len(valid_car_long_plate_images) = 978
len(valid_car_long_plate_labels) = 978


In [25]:
images_destination_folder = destination_folder + '/images'
labels_destination_folder = destination_folder + '/labels'

if not os.path.exists(images_destination_folder):
    os.makedirs(images_destination_folder)
    
if not os.path.exists(labels_destination_folder):
    os.makedirs(labels_destination_folder)

for long_plate_image, long_plate_label in zip(valid_car_long_plate_images, valid_car_long_plate_labels):
    source_image_path = os.path.join(valid_images_folder, long_plate_image)
    destination_image_path = os.path.join(images_destination_folder, long_plate_image)
    
    source_label_path = os.path.join(valid_labels_folder, long_plate_label)
    destination_label_path = os.path.join(labels_destination_folder, long_plate_label)
    
    shutil.copy(source_image_path, destination_image_path)
    shutil.copy(source_label_path, destination_label_path)

print("Copy process completed")


Copy process completed


In [26]:
long_plate_images = os.listdir(images_destination_folder)
long_plate_labels = os.listdir(labels_destination_folder)

print(f"len(long_plate_images) = {len(long_plate_images)}")
print(f"len(long_plate_labels) = {len(long_plate_labels)}")

len(long_plate_images) = 978
len(long_plate_labels) = 978


## Test files

In [27]:
source_folder = 'data/Vehicle Registration Plates.v1-original-images.yolov8/test'
destination_folder = 'data/1_chosen_data/test'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

test_images_folder = source_folder + '/images'
test_labels_folder = source_folder + '/labels'
test_images = os.listdir(test_images_folder)
test_labels = os.listdir(test_labels_folder)

print(f"len(test_images) = {len(test_images)}")
print(f"len(test_labels) = {len(test_labels)}")

len(test_images) = 882
len(test_labels) = 882


In [28]:
test_car_long_plate_images = [image_name for image_name in test_images if image_name.startswith('CarLongPlate')]
test_car_long_plate_labels = [label for label in test_labels if label.startswith('CarLongPlate')]

print(f"len(test_car_long_plate_images) = {len(test_car_long_plate_images)}")
print(f"len(test_car_long_plate_labels) = {len(test_car_long_plate_labels)}")

len(test_car_long_plate_images) = 490
len(test_car_long_plate_labels) = 490


In [31]:
images_destination_folder = destination_folder + '/images'
labels_destination_folder = destination_folder + '/labels'

if not os.path.exists(images_destination_folder):
    os.makedirs(images_destination_folder)
    
if not os.path.exists(labels_destination_folder):
    os.makedirs(labels_destination_folder)

for long_plate_image, long_plate_label in zip(test_car_long_plate_images, test_car_long_plate_labels):
    source_image_path = os.path.join(test_images_folder, long_plate_image)
    destination_image_path = os.path.join(images_destination_folder, long_plate_image)
    
    source_label_path = os.path.join(test_labels_folder, long_plate_label)
    destination_label_path = os.path.join(labels_destination_folder, long_plate_label)
    
    shutil.copy(source_image_path, destination_image_path)
    shutil.copy(source_label_path, destination_label_path)

print("Copy process completed")

Copy process completed


In [32]:
long_plate_images = os.listdir(images_destination_folder)
long_plate_labels = os.listdir(labels_destination_folder)

print(f"len(long_plate_images) = {len(long_plate_images)}")
print(f"len(long_plate_labels) = {len(long_plate_labels)}")

len(long_plate_images) = 490
len(long_plate_labels) = 490


# Change the contents of the labels folder to match the contents of the images after removing the broken images

## Train files

In [5]:
images_folder = 'data/1_selected_data/train/images'
labels_folder = 'data/1_selected_data/train/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(train_images) = {len(image_files)}")
print(f"len(train_labels) = {len(label_files)}")

len(train_images) = 2715
len(train_labels) = 3436


In [7]:
test_label = label_files[0]
image_filename = os.path.splitext(test_label)[0] + '.jpg'
image_filename

'CarLongPlate0_jpg.rf.68e0b9ccbdf0663f58876e02d480971e.jpg'

In [ ]:
image_files[0] == image_filename

True

In [10]:
# Loop through each label file and check if the corresponding image exists
for label_file in label_files:
    if label_file.endswith('.txt'):
        # Extract the image filename without the extension
        image_filename = os.path.splitext(label_file)[0] + '.jpg'

        # Check if the corresponding image exists in the 'images' folder
        if image_filename not in image_files:
            # If the image does not exist, delete the label file
            label_file_path = os.path.join(labels_folder, label_file)
            os.remove(label_file_path)

print("Deletion process completed")

Deletion process completed.


In [11]:
images_folder = 'data/1_selected_data/train/images'
labels_folder = 'data/1_selected_data/train/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(train_images) = {len(image_files)}")
print(f"len(train_labels) = {len(label_files)}")

len(train_images) = 2715
len(train_labels) = 2715


## Valid files

In [12]:
images_folder = 'data/1_selected_data/valid/images'
labels_folder = 'data/1_selected_data/valid/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(valid_images) = {len(image_files)}")
print(f"len(valid_labels) = {len(label_files)}")

len(valid_images) = 796
len(valid_labels) = 978


In [13]:
# Loop through each label file and check if the corresponding image exists
for label_file in label_files:
    if label_file.endswith('.txt'):
        # Extract the image filename without the extension
        image_filename = os.path.splitext(label_file)[0] + '.jpg'

        # Check if the corresponding image exists in the 'images' folder
        if image_filename not in image_files:
            # If the image does not exist, delete the label file
            label_file_path = os.path.join(labels_folder, label_file)
            os.remove(label_file_path)

print("Deletion process completed")

Deletion process completed


In [14]:
images_folder = 'data/1_selected_data/valid/images'
labels_folder = 'data/1_selected_data/valid/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(valid_images) = {len(image_files)}")
print(f"len(valid_labels) = {len(label_files)}")

len(valid_images) = 796
len(valid_labels) = 796


## Test files

In [15]:
images_folder = 'data/1_selected_data/test/images'
labels_folder = 'data/1_selected_data/test/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(test_images) = {len(image_files)}")
print(f"len(test_labels) = {len(label_files)}")

len(test_images) = 394
len(test_labels) = 490


In [16]:
# Loop through each label file and check if the corresponding image exists
for label_file in label_files:
    if label_file.endswith('.txt'):
        # Extract the image filename without the extension
        image_filename = os.path.splitext(label_file)[0] + '.jpg'

        # Check if the corresponding image exists in the 'images' folder
        if image_filename not in image_files:
            # If the image does not exist, delete the label file
            label_file_path = os.path.join(labels_folder, label_file)
            os.remove(label_file_path)

print("Deletion process completed")

images_folder = 'data/1_selected_data/test/images'
labels_folder = 'data/1_selected_data/test/labels'

# List all image files in the 'images' folder
image_files = os.listdir(images_folder)

# List all label files in the 'labels' folder
label_files = os.listdir(labels_folder)

print(f"len(test_images) = {len(image_files)}")
print(f"len(test_labels) = {len(label_files)}")

Deletion process completed
len(test_images) = 394
len(test_labels) = 394
